In [19]:
# Importation des bibliothèques
import os
import numpy as np
import cv2 as cv
import random
import pickle
import time
from datetime import timedelta
import math
# CNN
import tensorflow as tf
tf.config.run_functions_eagerly(True)
from tensorflow import keras
from keras.models import Sequential
from keras.layers import BatchNormalization
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Activation
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import Dense
from keras import backend as K
# Graph
import matplotlib
matplotlib.use('Agg')  # Use Agg backend for non-interactive plotting
import matplotlib.pyplot as plt
# Matrice de confusion
from mlxtend.plotting import plot_confusion_matrix
from sklearn.metrics import confusion_matrix

# Categories
CATEGORIES = ["ya", "yab", "yach", "yad", "yadd", "yae", "yaf", "yag", "yagh", 
              "yagw", "yah", "yahh", "yaj", "yak", "yakw", "yal", "yam", "yan", 
              "yaq", "yar", "yarr", "yas", "yass", "yat", "yatt", "yaw", "yax", 
              "yay", "yaz", "yazz", "yey", "yi", "yu"]
T_CATEGORIES = ['ⴰ', 'ⴱ', 'ⵛ', 'ⴷ', 'ⴹ', 'ⵄ', 'ⴼ', 'ⴳ', 'ⵖ', 'ⴳⵯ', 'ⵀ', 'ⵃ', 'ⵊ', 'ⴽ', 'ⴽⵯ',
                'ⵍ', 'ⵎ', 'ⵏ', 'ⵇ', 'ⵔ', 'ⵕ', 'ⵙ', 'ⵚ', 'ⵜ', 'ⵟ', 'ⵡ', 'ⵅ', 'ⵢ', 'ⵣ', 'ⵥ', 'ⴻ', 'ⵉ', 'ⵓ']

training_data = []  # List to contain processed training data
testing_data = []   # List to contain processed testing data

# Path configuration (replace with your actual dataset path)
chemin = "/Users/macbook/Desktop/M2/Prof M. Benaddy Deep Learning/Tp 4/Tifinagh/Dataset"  # Update this to your dataset path
if chemin == "":
    chemin = "path/to/your/dataset"  # Fallback placeholder (should be replaced)
DIR = os.path.join(chemin)
TRAIN_DATA_DIR = os.path.join(DIR, "train_data/")  # Changed from "training_data/" to "train_data/" based on MLP code
TEST_DATA_DIR = os.path.join(DIR, "test_data/")    # Changed from "testing_data/" to "test_data/" based on MLP code
MODEL_DIR = os.path.join(DIR, "model/")
TRAIN_MODEL_DIR = os.path.join(MODEL_DIR, "train/")
TEST_MODEL_DIR = os.path.join(MODEL_DIR, "test/")

# Debug print to verify path
print("Checking training data path:", TRAIN_DATA_DIR)
print("Available subdirectories:", [d for d in os.listdir(DIR) if os.path.isdir(os.path.join(DIR, d))])

# Ensure directories exist
os.makedirs(TRAIN_MODEL_DIR, exist_ok=True)
os.makedirs(TEST_MODEL_DIR, exist_ok=True)

# Prepare training data
def prepare_training_data():
    for category in CATEGORIES:
        path = os.path.join(TRAIN_DATA_DIR, category)
        if not os.path.exists(path):
            print(f"Directory not found: {path}")
            continue
        class_num = CATEGORIES.index(category)
        for img in os.listdir(path):
            if img.lower().endswith(('.png', '.jpg', '.jpeg')):
                img_array = cv.imread(os.path.join(path, img), cv.IMREAD_GRAYSCALE)
                if img_array is not None:
                    new_array = cv.resize(img_array, (50, 50))
                    training_data.append([new_array, class_num])

prepare_training_data()

# Prepare testing data
def prepare_testing_data():
    for category in CATEGORIES:
        path = os.path.join(TEST_DATA_DIR, category)
        if not os.path.exists(path):
            print(f"Directory not found: {path}")
            continue
        class_num = CATEGORIES.index(category)
        for img in os.listdir(path):
            if img.lower().endswith(('.png', '.jpg', '.jpeg')):
                img_array = cv.imread(os.path.join(path, img), cv.IMREAD_GRAYSCALE)
                if img_array is not None:
                    new_array = cv.resize(img_array, (50, 50))
                    testing_data.append([new_array, class_num])

prepare_testing_data()

# Shuffle data
random.shuffle(training_data)
random.shuffle(testing_data)

# Prepare training and testing sets
x_train = []
y_train = []
for features, label in training_data:
    x_train.append(features)
    y_train.append(label)

x_train = np.array(x_train)
y_train = np.array(y_train)

x_test = []
y_test = []
for features, label in testing_data:
    x_test.append(features)
    y_test.append(label)

x_test = np.array(x_test)
y_test = np.array(y_test)

# Save processed data
pickle_out = open(os.path.join(TEST_MODEL_DIR, "x_test.pickle"), "wb")
pickle.dump(x_test, pickle_out)
pickle_out.close()

pickle_out = open(os.path.join(TEST_MODEL_DIR, "y_test.pickle"), "wb")
pickle.dump(y_test, pickle_out)
pickle_out.close()

pickle_out = open(os.path.join(TRAIN_MODEL_DIR, "x_train.pickle"), "wb")
pickle.dump(x_train, pickle_out)
pickle_out.close()

pickle_out = open(os.path.join(TRAIN_MODEL_DIR, "y_train.pickle"), "wb")
pickle.dump(y_train, pickle_out)
pickle_out.close()

# Load processed data
pickle_in = open(os.path.join(TRAIN_MODEL_DIR, "x_train.pickle"), "rb")
x_train = pickle.load(pickle_in)

pickle_in = open(os.path.join(TRAIN_MODEL_DIR, "y_train.pickle"), "rb")
y_train = pickle.load(pickle_in)

pickle_in = open(os.path.join(TEST_MODEL_DIR, "x_test.pickle"), "rb")
x_test = pickle.load(pickle_in)

pickle_in = open(os.path.join(TEST_MODEL_DIR, "y_test.pickle"), "rb")
y_test = pickle.load(pickle_in)

# Image parameters
img_w = 50
img_h = 50
img_size_flat = img_w * img_h
img_shape = (img_w, img_h)
num_classes = 33
num_channels = 1

# Normalize and reshape data
x_train = x_train.reshape(-1, img_w, img_h, num_channels).astype("float32") / 255.0
x_test = x_test.reshape(-1, img_w, img_h, num_channels).astype("float32") / 255.0

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

# Define VGG16 model
def VGG16(width, height, depth, classes):
    model = Sequential()
    inputShape = (height, width, depth)
    chanDim = -1
    
    model.add(Conv2D(32, (3, 3), padding="same", input_shape=inputShape))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=chanDim))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(64, (3, 3), padding="same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=chanDim))
    model.add(Conv2D(64, (3, 3), padding="same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=chanDim))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.40))

    model.add(Conv2D(128, (3, 3), padding="same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=chanDim))
    model.add(Conv2D(128, (3, 3), padding="same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=chanDim))
    model.add(Conv2D(128, (3, 3), padding="same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=chanDim))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.40))

    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation("relu"))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))

    model.add(Dense(classes))
    model.add(Activation("softmax"))

    return model

# Training parameters
INIT_LR = 0.001
EPOCHS = 15
BS = 16

print("[INFO] training network...")
opt = keras.optimizers.Adam(learning_rate=INIT_LR, decay=INIT_LR / EPOCHS)
model = VGG16(img_w, img_h, num_channels, num_classes)
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
print("Model info...\n")
print(model.summary())

# Train the model
H = model.fit(x=x_train, y=y_train, batch_size=BS,
              validation_data=(x_test, y_test),
              steps_per_epoch=len(y_train) // BS, epochs=EPOCHS)
print("[INFO] training Done...")

# Save the model
model.save(os.path.join(MODEL_DIR, "VGG16-4.model"))
model.save('model4.h5')

# Convert to TensorFlow.js and zip
os.makedirs("model4", exist_ok=True)
!tensorflowjs_converter --input_format=tf_saved_model model4.h5 model4/
!zip -r model4.zip model4

# Load and evaluate the model
model = keras.models.load_model(os.path.join(MODEL_DIR, "VGG16-4.model"))
scores = model.evaluate(x_test, y_test, verbose=0)

# Plot training loss
N = np.arange(0, EPOCHS)
plt.style.use("ggplot")
plt.figure()
plt.plot(N, H.history["loss"], label="train_loss")
plt.plot(N, H.history["val_loss"], label="val_loss")
plt.title("Loss: VGG16 By HexaCoders")
plt.xlabel("Epoch #")
plt.ylabel("Loss")
plt.legend()
plt.savefig("Loss_plot_256.png")

# Plot training accuracy
N = np.arange(0, EPOCHS)
plt.style.use("ggplot")
plt.figure()
plt.plot(N, H.history["accuracy"], label="train_acc")
plt.plot(N, H.history["val_accuracy"], label="val_acc")
plt.title("Accuracy: VGG16 By HexaCoders")
plt.xlabel("Epoch #")
plt.ylabel("Accuracy")
plt.legend()
plt.savefig("Accuracy_plot_256.png")

# Plot confusion matrix
y_pred = np.argmax(model.predict(x_test), axis=1)
y_test_mat = np.argmax(y_test, axis=1)
mat = confusion_matrix(y_test_mat, y_pred)
plt.figure()
plot_confusion_matrix(conf_mat=mat, figsize=(14, 14), class_names=T_CATEGORIES)
plt.savefig("conf_mat.png")
print("Ploting Conf_Mat..................")
print(f"Accuracy = {scores[1]*100:.2f}%")


Checking training data path: /Users/macbook/Desktop/M2/Prof M. Benaddy Deep Learning/Tp 4/Tifinagh/Dataset/train_data/
Available subdirectories: ['test_data', 'train_data', 'model']
Directory not found: /Users/macbook/Desktop/M2/Prof M. Benaddy Deep Learning/Tp 4/Tifinagh/Dataset/train_data/yagw
Directory not found: /Users/macbook/Desktop/M2/Prof M. Benaddy Deep Learning/Tp 4/Tifinagh/Dataset/train_data/yakw
Directory not found: /Users/macbook/Desktop/M2/Prof M. Benaddy Deep Learning/Tp 4/Tifinagh/Dataset/test_data/yagw
Directory not found: /Users/macbook/Desktop/M2/Prof M. Benaddy Deep Learning/Tp 4/Tifinagh/Dataset/test_data/yakw
[INFO] training network...


/opt/anaconda3/lib/python3.12/site-packages/keras/src/optimizers/base_optimizer.py:86: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model info...



Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 50, 50, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 50, 50, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 50, 50, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 25, 25, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 25, 25, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_1 (Activation)       │ (None, 25, 25, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 25, 25, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 25, 25, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_2 (Activation)       │ (None, 25, 25, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 25, 25, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 12, 12, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 12, 12, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 12, 12, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_3 (Activation)       │ (None, 12, 12, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 12, 12, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 12, 12, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_4 (Activation)       │ (None, 12, 12, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 12, 12, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 12, 12, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_5 (Activation)       │ (None, 12, 12, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 12, 12, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 6, 6, 128)      │             

...

/opt/anaconda3/lib/python3.12/site-packages/tensorflow/python/data/ops/structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


Epoch 1/15
1654/1654 ━━━━━━━━━━━━━━━━━━━━ 1216s 735ms/step - accuracy: 0.5767 - loss: 1.5329 - val_accuracy: 0.9413 - val_loss: 0.1896
Epoch 2/15
   1/1654 ━━━━━━━━━━━━━━━━━━━━ 17:51 648ms/step - accuracy: 0.7000 - loss: 1.3600

/opt/anaconda3/lib/python3.12/site-packages/keras/src/trainers/epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


1654/1654 ━━━━━━━━━━━━━━━━━━━━ 202s 122ms/step - accuracy: 0.7000 - loss: 1.3600 - val_accuracy: 0.9453 - val_loss: 0.1787
Epoch 3/15
1654/1654 ━━━━━━━━━━━━━━━━━━━━ 1357s 821ms/step - accuracy: 0.9081 - loss: 0.2794 - val_accuracy: 0.9685 - val_loss: 0.1020
Epoch 4/15
1654/1654 ━━━━━━━━━━━━━━━━━━━━ 294s 178ms/step - accuracy: 1.0000 - loss: 0.0145 - val_accuracy: 0.9672 - val_loss: 0.1056
Epoch 5/15
1654/1654 ━━━━━━━━━━━━━━━━━━━━ 1428s 863ms/step - accuracy: 0.9384 - loss: 0.1918 - val_accuracy: 0.9552 - val_loss: 0.1462
Epoch 6/15
1654/1654 ━━━━━━━━━━━━━━━━━━━━ 189s 114ms/step - accuracy: 1.0000 - loss: 0.0227 - val_accuracy: 0.9538 - val_loss: 0.1538
Epoch 7/15
1654/1654 ━━━━━━━━━━━━━━━━━━━━ 1296s 784ms/step - accuracy: 0.9559 - loss: 0.1351 - val_accuracy: 0.9891 - val_loss: 0.0389
Epoch 8/15
1654/1654 ━━━━━━━━━━━━━━━━━━━━ 221s 133ms/step - accuracy: 1.0000 - loss: 0.0098 - val_accuracy: 0.9892 - val_loss: 0.0383
Epoch 9/15
1654/1654 ━━━━━━━━━━━━━━━━━━━━ 1521s 919ms/step - accuracy:

ValueError: Invalid filepath extension for saving. Please add either a `.keras` extension for the native Keras format (recommended) or a `.h5` extension. Use `model.export(filepath)` if you want to export a SavedModel for use with TFLite/TFServing/etc. Received: filepath=/Users/macbook/Desktop/M2/Prof M. Benaddy Deep Learning/Tp 4/Tifinagh/Dataset/model/VGG16-4.model.